**Step 1: Data Loading and Initial Exploration**


In [1]:
import pandas as pd

file_path = "Hotels_data_Changed.csv"
df = pd.read_csv(file_path)

print(df.head())
print(df.info())
print(df.isnull().sum())
print(df.describe())

   Unnamed: 0  Snapshot ID   Snapshot Date    Checkin Date  Days  \
0           0            1  7/17/2015 0:00  8/12/2015 0:00     5   
1           1            1  7/17/2015 0:00  8/19/2015 0:00     5   
2           2            1  7/17/2015 0:00  8/13/2015 0:00     5   
3           3            1  7/17/2015 0:00  7/26/2015 0:00     5   
4           4            1  7/17/2015 0:00  8/12/2015 0:00     5   

   Original Price  Discount Price  Discount Code  Available Rooms  \
0            1178            1040              1                6   
1            1113             982              1                8   
2            4370            4240              1                3   
3            1739            1667              1               18   
4            1739            1672              1                3   

                               Hotel Name  Hotel Stars  DayDiff  WeekDay  \
0  Best Western Plus Seaport Inn Downtown            3  26 days        2   
1  Best Western Plus Sea

# **Step 2: Date Processing and Feature Engineering**  

To enable accurate date-based calculations, we convert the **Snapshot Date** and **Check-in Date** columns into datetime format.  
From these, we extract three new features that provide important temporal insights for pricing predictions:  

- **Days Until Check-in** – The number of days between `Snapshot Date` (when the price was recorded) and `Checkin Date` (when the booking occurs).  
  - This feature helps capture how prices change over time as the check-in date approaches.  

- **Snapshot Month** – Extracted as the month from `Snapshot Date`.  
  - This allows the model to learn seasonal pricing trends.  

- **Check-in Month** – Extracted as the month from `Checkin Date`.  
  - This helps the model understand peak travel months and demand fluctuations.  

By incorporating these engineered date features, we align with the requirement to extract meaningful temporal patterns from the dataset, ultimately **improving model performance**.  


In [2]:
df["Snapshot Date"] = pd.to_datetime(df["Snapshot Date"])
df["Checkin Date"] = pd.to_datetime(df["Checkin Date"])

df["Days Until Checkin"] = (df["Checkin Date"] - df["Snapshot Date"]).dt.days
df["Snapshot Month"] = df["Snapshot Date"].dt.month
df["Checkin Month"] = df["Checkin Date"].dt.month

print(df[["Snapshot Date", "Checkin Date", "Days Until Checkin", "Snapshot Month", "Checkin Month"]].head())
print(df.info())


  Snapshot Date Checkin Date  Days Until Checkin  Snapshot Month  \
0    2015-07-17   2015-08-12                  26               7   
1    2015-07-17   2015-08-19                  33               7   
2    2015-07-17   2015-08-13                  27               7   
3    2015-07-17   2015-07-26                   9               7   
4    2015-07-17   2015-08-12                  26               7   

   Checkin Month  
0              8  
1              8  
2              8  
3              7  
4              8  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68175 entries, 0 to 68174
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          68175 non-null  int64         
 1   Snapshot ID         68175 non-null  int64         
 2   Snapshot Date       68175 non-null  datetime64[ns]
 3   Checkin Date        68175 non-null  datetime64[ns]
 4   Days                68175 n

# **Step 3: Data Filtering and Outlier Removal**  

To ensure **data consistency and realistic price values**, the dataset is filtered, and extreme outliers are removed.  
The key steps in this process are:  

###**Filtering Valid Discount Prices**  
- Removing cases where the `Discount Price` is **greater than or equal to the `Original Price`**, as these entries are invalid.  

###**Outlier Removal Using Percentile Filtering**  
- We exclude extreme values based on statistical thresholds:  
  - **1st percentile (0.01 quantile)** → Removing unusually low prices.  
  - **99th percentile (0.99 quantile)** → Removing extremely high prices.  
- This filtering is applied **separately** to both `Original Price` and `Discount Price`, ensuring that pricing values remain within a reasonable range.  

By performing this step, we maintain a **clean and reliable dataset**, removing unrealistic price points and ensuring the model learns from meaningful, high-quality data.  


In [3]:
df_filtered = df[["Snapshot Date", "Hotel Name", "Discount Code", "Checkin Date", "Original Price", "Discount Price"]]

df_filtered = df_filtered[df_filtered["Discount Price"] < df_filtered["Original Price"]]

q1 = df_filtered["Original Price"].quantile(0.01)
q3 = df_filtered["Original Price"].quantile(0.99)
df_filtered = df_filtered[(df_filtered["Original Price"] >= q1) & (df_filtered["Original Price"] <= q3)]

q1 = df_filtered["Discount Price"].quantile(0.01)
q3 = df_filtered["Discount Price"].quantile(0.99)
df_filtered = df_filtered[(df_filtered["Discount Price"] >= q1) & (df_filtered["Discount Price"] <= q3)]

print(df_filtered.describe())
print(df_filtered.info())


                       Snapshot Date  Discount Code  \
count                          65541   65541.000000   
mean   2015-08-25 06:11:22.528493824       2.425428   
min              2015-07-17 00:00:00       1.000000   
25%              2015-08-03 00:00:00       2.000000   
50%              2015-08-27 00:00:00       2.000000   
75%              2015-09-13 00:00:00       3.000000   
max              2015-09-30 00:00:00       4.000000   
std                              NaN       1.007606   

                        Checkin Date  Original Price  Discount Price  
count                          65541    65541.000000    65541.000000  
mean   2015-09-11 23:57:49.492378624     1946.028211     1793.188096  
min              2015-07-18 00:00:00      655.000000      615.000000  
25%              2015-08-24 00:00:00     1346.000000     1230.000000  
50%              2015-09-13 00:00:00     1795.000000     1651.000000  
75%              2015-10-01 00:00:00     2304.000000     2119.000000  
max    

# **Step 4: Data Filtering Based on Minimum Sample Size**  

In this step, we ensure that the dataset contains **sufficient data points** for each hotel and discount code combination.  
To achieve this, we applied the following filtering criteria:  

- **Minimum Sample Count:** A threshold of **10 samples per month** was set to retain only hotels and discount codes with enough historical data.  
- **Grouping Strategy:** Data was grouped by **Hotel Name, Discount Code, and Snapshot Month**, and only groups meeting the minimum sample requirement were kept.  
- **Before Filtering:** The dataset contained **`len(df)`** records.  
- **After Filtering:** The dataset was reduced to **`len(df_filtered)`** records, ensuring that only statistically meaningful data points were retained.  

This filtering step helps eliminate noisy or insufficient data points, leading to a **more reliable and robust model** in the next steps.  


In [4]:
min_samples = 10

df["Sample Count"] = df.groupby(["Hotel Name", "Discount Code", "Snapshot Month"])["Snapshot Date"].transform("count")

df_filtered = df[df["Sample Count"] >= min_samples]

print(df_filtered[["Hotel Name", "Discount Code", "Snapshot Month", "Sample Count"]].drop_duplicates().head())
print("Before filtering:", len(df))
print("After filtering:", len(df_filtered))
print(df_filtered.groupby(["Hotel Name", "Discount Code", "Snapshot Month"])["Snapshot Date"].count().describe())



                                           Hotel Name  Discount Code  \
2                              The Peninsula New York              1   
3                        Eventi Hotel a Kimpton Hotel              1   
5                                Grand Hyatt New York              1   
10  DoubleTree Suites by Hilton New York City - Ti...              1   
12                         Hampton Inn Manhattan Soho              1   

    Snapshot Month  Sample Count  
2                7            28  
3                7            16  
5                7            15  
10               7            22  
12               7            27  
Before filtering: 68175
After filtering: 63383
count    1192.000000
mean       53.173658
std        49.173719
min        10.000000
25%        19.000000
50%        37.000000
75%        70.000000
max       338.000000
Name: Snapshot Date, dtype: float64


# **Step 5: Selecting the Top 40 Hotels and Filtering Data**  

To improve the reliability of our model, we focused on the **40 hotels with the highest number of samples**.  
This ensures that predictions are based on well-represented data, reducing the impact of outliers or sparse records.  

The filtering process included the following steps:  
- **Selecting the Top 40 Hotels:** We identified the **40 hotels with the most records** in the dataset.  
- **Filtering for Relevant Check-in Dates:** Since the task requires predictions for **up to one month in advance**, we kept only records where `Days Until Checkin` is between **1 and 30 days**.  

### **Dataset Reduction Results**  
- **Number of samples after initial filtering:** `{df_filtered.shape[0]}`  
- **Number of samples for the top 40 hotels:** `{df_top_hotels.shape[0]}`  

By focusing on well-represented hotels and relevant time frames, we ensure that the model is trained on **high-quality and meaningful data**, leading to better predictions.  


In [5]:
top_hotels = df_filtered["Hotel Name"].value_counts().head(40).index
df_top_hotels = df_filtered[df_filtered["Hotel Name"].isin(top_hotels)]
df_top_hotels = df_top_hotels[(df_top_hotels["Days Until Checkin"] >= 1) & (df_top_hotels["Days Until Checkin"] <= 30)]


print(f"Number of samples after filtering: {df_filtered.shape[0]}")
print(f"Number of samples for top 40 hotels: {df_top_hotels.shape[0]}")


Number of samples after filtering: 63383
Number of samples for top 40 hotels: 34972


# **Step 6: Feature Engineering – Discount Calculation & Categorical Encoding**  

To improve the model, we engineered additional features that enhance prediction accuracy:  

- **Discount Percentage** – Calculated as:  
  \[
  \text{Discount Percentage} = \frac{\text{Original Price} - \text{Discount Price}}{\text{Original Price}}
  \]
  This represents the relative discount applied to each hotel booking.  

- **Snapshot & Check-in Weekday** – Extracted from the respective dates, allowing the model to capture weekly pricing patterns.  

- **Categorical Encoding** – Since machine learning models require numerical inputs, the categorical variables:  
  - `Hotel Name`  
  - `Discount Code`  
  were converted into numerical codes using **categorical encoding**.  

This step ensures that the dataset is **numerically structured** while preserving essential pricing patterns, leading to a more **robust and effective predictive model**.  


In [6]:
import pandas as pd

df_filtered = df_filtered.copy()

df_filtered.loc[:, "Discount Percentage"] = (df_filtered["Original Price"] - df_filtered["Discount Price"]) / df_filtered["Original Price"]

df_filtered.loc[:, "Snapshot Weekday"] = df_filtered["Snapshot Date"].dt.weekday
df_filtered.loc[:, "Checkin Weekday"] = df_filtered["Checkin Date"].dt.weekday

df_filtered.loc[:, "Hotel Name"] = df_filtered["Hotel Name"].astype("category").cat.codes
df_filtered.loc[:, "Discount Code"] = df_filtered["Discount Code"].astype("category").cat.codes

print(df_filtered.head())
print(df_filtered.info())


   Unnamed: 0  Snapshot ID Snapshot Date Checkin Date  Days  Original Price  \
2           2            1    2015-07-17   2015-08-13     5            4370   
3           3            1    2015-07-17   2015-07-26     5            1739   
4           4            1    2015-07-17   2015-08-12     5            1739   
5           5            1    2015-07-17   2015-08-07     5            1436   
6           6            1    2015-07-17   2015-08-09     5            1616   

   Discount Price  Discount Code  Available Rooms Hotel Name  ...  WeekDay  \
2            4240              0                3        172  ...        3   
3            1667              0               18         37  ...        6   
4            1672              0                3         37  ...        2   
5            1345              0                9         47  ...        4   
6            1525              0                5         47  ...        6   

  DiscountDiff  DiscountPerc  Days Until Checkin  Snapsh

# **Step 7: Splitting Data into Training and Testing Sets**

At this stage, the dataset is divided into training and testing sets to evaluate the model’s performance. To ensure a fair and balanced learning process, we applied the following steps:

- **Feature selection (X):** We excluded non-predictive columns such as `Snapshot Date` and `Checkin Date`, as they are not needed for predicting the target variable. The target variable is `Discount Price`.
- **Target variable (y):** The model predicts `Discount Price`.
- **Data split:**
  - **80% training, 20% testing** → Ensures a well-balanced dataset for learning and evaluation.
  - **Fixed random state (42)** → Guarantees reproducibility of results.

## Dataset Split Results:

- **Training Set Size:** {X_train.shape}
- **Testing Set Size:** {X_test.shape}

This step successfully prepares the dataset for **model training and performance evaluation**.


In [20]:
from sklearn.model_selection import train_test_split

X = df_top_hotels.drop(columns=["Snapshot Date", "Checkin Date", "Discount Price"])
y = df_top_hotels["Discount Price"]

X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.preprocessing import LabelEncoder

hotel_encoder = LabelEncoder()
discount_encoder = LabelEncoder()

df_top_hotels["Hotel Name"] = hotel_encoder.fit_transform(df_top_hotels["Hotel Name"])
df_top_hotels["Discount Code"] = discount_encoder.fit_transform(df_top_hotels["Discount Code"])


# **Step 8: Model Evaluation and Performance Analysis**

In this step, we evaluate the performance of the **Random Forest Regression model**, which was selected as the final model due to its superior performance.

The model was trained using a filtered dataset that meets the following criteria:

- **Minimum Sample Size:** The dataset includes only hotel and discount code combinations that have at least **10 samples per month**, ensuring reliable predictions.
- **Prediction Window:** The model was trained to predict discount prices for check-in dates **up to one month in advance**, in alignment with the problem's requirements.
- **Feature Selection:** The dataset was carefully processed, removing unnecessary columns while retaining meaningful features such as `Hotel Name`, `Discount Code`, `Original Price`, and `Days Until Check-in`.

## **Performance Metrics**

To evaluate the model, we used two key performance metrics:

- **Mean Absolute Error (MAE):** Measures the average absolute difference between predicted and actual discount prices. A lower value indicates better accuracy.
- **R² Score (Coefficient of Determination):** Measures how well the model explains the variance in discount prices, where a score closer to 1 indicates a better fit.

### **Final Model Performance:**

- **Mean Absolute Error (MAE):** **5.14 USD**
- **R² Score:** **0.9385**

These results indicate that the **Random Forest model provides highly accurate predictions**, with a minimal average error of **5.14 USD** and an **R² score of 0.9385**, meaning the model explains **93.85% of the variance** in discount prices.

 However, given the current results, this model successfully meets the project’s requirements for **accurate discount price predictions**.


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

X_train_numeric = X_train_top.select_dtypes(include=['number'])
X_test_numeric = X_test_top.select_dtypes(include=['number'])

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

model_rf.fit(X_train_scaled, y_train_top)

y_pred_rf = model_rf.predict(X_test_scaled)

mae_rf = mean_absolute_error(y_test_top, y_pred_rf)
r2_rf = r2_score(y_test_top, y_pred_rf)

print(f"Mean Absolute Error (Random Forest): {mae_rf:.2f}")
print(f"R2 Score (Random Forest): {r2_rf:.4f}")


Mean Absolute Error (Random Forest): 5.14
R2 Score (Random Forest): 0.9385
